In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
!ls "/content/gdrive/My Drive/"

'Colab Notebooks'   Dataset


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import turicreate as tc

In [35]:
events=pd.read_csv('/content/drive/My Drive/Colab Notebooks/events.csv')
events2=events
events.event.unique()

array(['view', 'addtocart', 'transaction', 'add'], dtype=object)

In [36]:
events2['event']=events2['event'].replace([ 'addtocart', 'transaction', 'add'],'view',inplace=False)
events2

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908.0,NaN
1,1433224214164,992329,view,248676.0,NaN
2,1433221999827,111016,view,318965.0,NaN
3,1433221955914,483717,view,253185.0,NaN
4,1433221337106,951259,view,367447.0,NaN
...,...,...,...,...,...
2545358,1437704763808,878862,view,6552.0,NaN
2545359,1437704509585,828962,view,49461.0,NaN
2545360,1437703586293,1067033,view,150212.0,NaN
2545361,1437702581129,1208743,view,379387.0,NaN


In [0]:
events2['event'].unique()
events=events2

In [0]:
events=events[events.event != 'addtocart']
events=events[events.event != 'transaction']
events=events[events.event != 'add']

In [0]:
events=events.drop(['timestamp','transactionid'],axis=1)

In [0]:
events["purchase_count"]=''
events.head()
events.columns = events.columns.str.strip()
events.to_csv('test3.csv')

In [41]:
events['visitorid'].value_counts()
events.loc[events['visitorid'] == 1150086]


,visitorid,event,itemid,purchase_count
226726,1150086,view,49925.0,
226815,1150086,view,332592.0,
226997,1150086,view,408132.0,
227022,1150086,view,109352.0,
227094,1150086,view,107195.0,
...,...,...,...,...
2543713,1150086,view,339703.0,
2544228,1150086,view,357132.0,
2544250,1150086,view,304877.0,
2544383,1150086,view,300314.0,


In [27]:
group=events.groupby('visitorid')
df3 = group.apply(lambda x: x['itemid'].unique())
#df2 = pd.DataFrame({'visitorid':events.visitorid.unique()})
df3

visitorid
0                     [285930.0, 357564.0, 67045.0]
1                                         [72028.0]
2          [216305.0, 325215.0, 342816.0, 259884.0]
3                                        [385090.0]
4                                        [177677.0]
                             ...                   
1407575                                  [121220.0]
1407576                                  [356208.0]
1407577                                  [427784.0]
1407578                                  [188736.0]
1407579                                    [2521.0]
Length: 1308174, dtype: object

In [34]:
events

,visitorid,event,itemid,purchase_count
0,257597,view,355908.0,
1,992329,view,248676.0,
2,111016,view,318965.0,
3,483717,view,253185.0,
4,951259,view,367447.0,
...,...,...,...,...
2545358,878862,view,6552.0,
2545359,828962,view,49461.0,
2545360,1067033,view,150212.0,
2545361,1208743,view,379387.0,


In [28]:
#df4=pd.DataFrame({'visitorid':df2.index, 'products':df2.values})
df4=df3
df4=df4.to_frame()
df4.columns=['productid']
df4['customerid'] = df4.index
df4=df4.reset_index(drop=True)
df4=df4.rename(columns={ df4.columns[0]: "products" })
df4=df4.rename(columns={ df4.columns[1]: "customerId" })
df4.loc[df4['customerId'] == 1150086]
#df5=df4.products.apply(lambda x: x.replace(',','|'))

,products,customerId
1068695,"[49925.0, 332592.0, 408132.0, 109352.0, 107195...",1150086


In [0]:
df4['products']=df4.products.apply(Counter)

In [13]:
df4.loc[df4['customerId']==3]

,products,customerId
3,[385090.0],3


In [0]:
new_data=pd.DataFrame()
data_split=np.array_split(df4,50)
for each in data_split:
  data = pd.melt(each.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
  data['productId'] = data['productId'].astype(np.int64)
  new_data = new_data.append(data, ignore_index=True)



In [30]:
new_data
#new_data.to_csv('input.csv')
#new_data

,customerId,productId,purchase_count
0,0,67045,1
1,0,285930,1
2,0,357564,1
3,1,72028,1
4,2,216305,1
...,...,...,...
1982362,1407575,121220,1
1982363,1407576,356208,1
1982364,1407577,427784,1
1982365,1407578,188736,1


In [0]:
#df5=events.groupby('visitorid').itemid.agg(['count', 'first'])
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(new_data)


In [0]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

train_data_dummy, test_data_dummy = split_data(data_dummy)

In [0]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(events['visitorid'])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
  model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')

In [48]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1585893 observations with 1092620 users and 211536 items.

Data prepared in: 1.46321s

...

| 7.06s                               | 100              | 211536          |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 8.25354s

In [52]:
models_w_dummy = [ cos_dummy]
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
eval_dummy = tc.recommender.util.compare_models(test_data_dummy,cos_dummy, model_names='Cosine similarity')

TypeError: ignored

In [0]:
chunker = pd.read_csv('/content/test4.csv', chunksize=1000)
tot=pd.DataFrame();
for piece in chunker:
  tot=tot.add(piece.reset_index().pivot(values='prod_freq',columns='itemid',index='customerid'),fill_value=0)

   

In [0]:
df5.columns = df5.columns.str.strip()
print (df5.columns.tolist())
df5.customerid.unique()
df5.to_csv('test2.csv')

NameError: ignored